In [ ]:
import pandas as pd
import numpy as np 
from sklearn.ensemble import RandomForestRegressor

In [ ]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_colwidth', None)

In [ ]:
# prepare training and test data.
# 2011-01-29 ~ 2016-04-24 : d_1    ~ d_1913
# 2016-04-25 ~ 2016-05-22 : d_1914 ~ d_1941 (public)
# 2016-05-23 ~ 2016-06-19 : d_1942 ~ d_1969 (private)

test = pd.read_csv('inputs/sample_submission.csv')
test2 = test[30490:]
test = test[:30490]
test = pd.melt(test, id_vars=['id'], var_name='d', value_name='target')
for i in range(1, 29):
    test = test.replace({f'F{i}': f'd_{1913+i}'})
    
calendar = pd.read_csv('inputs/calendar.csv')
date = calendar[['d', 'date', 'wm_yr_wk', 'wday','month','year']]

train = pd.read_csv('inputs/sales_train_validation.csv')
train = pd.melt(train, id_vars=['id','item_id','dept_id','cat_id','store_id','state_id'], var_name='d', value_name='target')

sell_prices = pd.read_csv('inputs/sell_prices.csv')

In [ ]:
train.head()

In [ ]:
train = train.merge(date, how='left')

train['day'] = train['date'].astype('datetime64').dt.day
train['week'] = train['date'].astype('datetime64').dt.week


In [ ]:
test[['cat_id', 'dept_id', 'item_id', 'state_id', 'store_id', 'tmp']] = pd.DataFrame(test['id'].str.split('_').tolist())
del test['tmp']
test['store_id'] = test['state_id'] + '_' + test['store_id']
test['dept_id'] = test['cat_id'] + '_' + test['dept_id']
test['item_id'] = test['dept_id'] + '_' + test['item_id']

In [ ]:
test = test.merge(date, how='left')
test['day'] = test['date'].astype('datetime64').dt.day
test['week'] = test['date'].astype('datetime64').dt.week


In [ ]:
test = test[train.columns]

In [ ]:
train.head()

In [ ]:
all_df = pd.concat([train, test])

del all_df['date']


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for i in all_df.columns[all_df.dtypes == 'object']:
    all_df[i] = le.fit_transform(list(all_df[i]))

In [ ]:
all_df.head()

In [ ]:
train2 = all_df[:len(train)]
train2 = train2.drop('target', axis=1)

In [ ]:
test2 = all_df[len(train):]
test2 = test2.drop('target', axis=1)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100, n_jobs=-1)
rf.fit(train2, np.log(train['target'] + 1))
result = rf.predict(test2)